# Chapter 28: Other CI/CD Tools

While Chapters 25-27 examined Jenkins, GitHub Actions, and GitLab CI/CD as the dominant platforms, the CI/CD ecosystem encompasses numerous specialized tools optimized for specific cloud environments, architectural patterns, or organizational requirements. This chapter explores alternative platforms that excel in particular contexts—CircleCI's developer experience optimizations, Azure DevOps' enterprise integration, AWS CodePipeline's native cloud orchestration, Google Cloud Build's serverless execution, Tekton's Kubernetes-native approach, and Concourse CI's resource-oriented philosophy.

Understanding these alternatives enables informed platform selection based on existing infrastructure investments, compliance constraints, and specific workflow requirements, while migration strategies facilitate transitions between platforms as organizational needs evolve.

## 28.1 CircleCI

CircleCI pioneered cloud-native continuous integration with a focus on developer experience, fast feedback loops, and sophisticated caching mechanisms. Operating as a managed service (with self-hosted runner options), CircleCI emphasizes speed through parallelization and intelligent resource allocation.

### Configuration Architecture

CircleCI uses YAML configuration (`.circleci/config.yml`) with a job-based structure emphasizing workflows and orbs (reusable packages):

```yaml
# .circleci/config.yml
version: 2.1

# Orbs: Reusable packages of configuration
orbs:
  node: circleci/node@5.1.0
  docker: circleci/docker@2.6.0
  kubernetes: circleci/kubernetes@1.3.1
  sonarcloud: sonarsource/sonarcloud@2.0.0

# Executors: Define the execution environment
executors:
  default:
    docker:
      - image: cimg/node:20.0
    resource_class: medium
  with-postgres:
    docker:
      - image: cimg/node:20.0
      - image: cimg/postgres:15.0
        environment:
          POSTGRES_USER: postgres
          POSTGRES_PASSWORD: postgres
          POSTGRES_DB: test_db

# Commands: Reusable command sequences
commands:
  setup-npm:
    steps:
      - restore_cache:
          keys:
            - npm-v1-{{ checksum "package-lock.json" }}
            - npm-v1-
      - run:
          name: Install dependencies
          command: npm ci
      - save_cache:
          key: npm-v1-{{ checksum "package-lock.json" }}
          paths:
            - ~/.npm

  deploy-to-k8s:
    parameters:
      namespace:
        type: string
      image-tag:
        type: string
    steps:
      - kubernetes/install-kubectl
      - run:
          name: Deploy to Kubernetes
          command: |
            kubectl set image deployment/app app=<< parameters.image-tag >> -n << parameters.namespace >>
            kubectl rollout status deployment/app -n << parameters.namespace >>

# Jobs: Individual units of work
jobs:
  build-and-test:
    executor: default
    steps:
      - checkout
      - setup-npm
      - run:
          name: Run tests
          command: npm test -- --coverage
      - store_test_results:
          path: test-results
      - store_artifacts:
          path: coverage
          destination: coverage-reports
      - persist_to_workspace:
          root: .
          paths:
            - dist
            - node_modules

  security-scan:
    executor: default
    steps:
      - checkout
      - setup-npm
      - run:
          name: Security audit
          command: npm audit --audit-level=high
      - sonarcloud/scan

  build-docker:
    executor: docker/docker
    steps:
      - checkout
      - setup_remote_docker:
          docker_layer_caching: true  # Paid feature, speeds up builds
      - run:
          name: Build Docker image
          command: |
            docker build -t $DOCKER_REGISTRY/app:${CIRCLE_SHA1} .
            docker push $DOCKER_REGISTRY/app:${CIRCLE_SHA1}

  deploy-staging:
    executor: default
    steps:
      - checkout
      - attach_workspace:
          at: .
      - deploy-to-k8s:
          namespace: staging
          image-tag: $DOCKER_REGISTRY/app:${CIRCLE_SHA1}

  deploy-production:
    executor: default
    steps:
      - checkout
      - deploy-to-k8s:
          namespace: production
          image-tag: $DOCKER_REGISTRY/app:${CIRCLE_SHA1}

# Workflows: Orchestrate job execution
workflows:
  version: 2
  build-test-deploy:
    jobs:
      - build-and-test
      - security-scan:
          requires:
            - build-and-test
          filters:
            branches:
              only: main
      - build-docker:
          requires:
            - build-and-test
          filters:
            branches:
              only: main
      - deploy-staging:
          requires:
            - build-docker
            - security-scan
      - hold-for-approval:
          type: approval
          requires:
            - deploy-staging
          filters:
            branches:
              only: main
      - deploy-production:
          requires:
            - hold-for-approval
```

### Key Differentiators

**Resource Classes:**
CircleCI offers granular resource allocation from small (1 vCPU, 2GB RAM) to 2XL+ (20 vCPU, 40GB RAM) and GPU instances for machine learning workloads:

```yaml
jobs:
  ml-training:
    docker:
      - image: cimg/python:3.11
    resource_class: gpu.nvidia.medium  # GPU-enabled runner
    steps:
      - run: python train.py
```

**Test Splitting:**
Intelligent test parallelization based on timing data:

```yaml
jobs:
  test:
    parallelism: 4  # Split across 4 containers
    steps:
      - run:
          command: |
            circleci tests glob "tests/**/*.test.js" | \
            circleci tests split --split-by=timings | \
            xargs npm test
```

**Insights and Monitoring:**
Built-in dashboard tracking credit usage, success rates, and pipeline duration trends with recommendations for optimization.

## 28.2 Azure DevOps Pipelines

Microsoft Azure DevOps Pipelines integrates tightly with the Azure ecosystem, offering robust enterprise features, ARM template deployment, and comprehensive artifact management through Azure Artifacts.

### YAML Schema Structure

Azure Pipelines uses a hierarchical YAML structure emphasizing stages, jobs, and steps with strong typing and template support:

```yaml
# azure-pipelines.yml
trigger:
  branches:
    include:
      - main
      - release/*
  paths:
    exclude:
      - docs/*
      - '*.md'

pr:
  branches:
    include:
      - main

variables:
  - group: production-variables  # Reference variable groups
  - name: dockerRegistry
    value: 'myregistry.azurecr.io'
  - name: imageRepository
    value: 'myapp'
  - name: tag
    value: '$(Build.BuildId)'

stages:
- stage: Build
  displayName: 'Build and Test'
  jobs:
  - job: BuildAndTest
    displayName: 'Build & Test'
    pool:
      vmImage: 'ubuntu-latest'
    steps:
    - task: NodeTool@0
      inputs:
        versionSpec: '20.x'
      displayName: 'Install Node.js'

    - script: npm ci
      displayName: 'Install dependencies'

    - script: npm run lint
      displayName: 'Run linting'

    - script: npm run build
      displayName: 'Build application'

    - task: PublishTestResults@2
      condition: succeededOrFailed()
      inputs:
        testRunner: JUnit
        testResultsFiles: '**/test-results.xml'

    - task: PublishCodeCoverageResults@1
      inputs:
        codeCoverageTool: Cobertura
        summaryFileLocation: '$(System.DefaultWorkingDirectory)/**/coverage/cobertura-coverage.xml'

    - task: PublishBuildArtifacts@1
      inputs:
        PathtoPublish: '$(Build.ArtifactStagingDirectory)'
        ArtifactName: 'drop'
        publishLocation: 'Container'

- stage: SecurityScan
  displayName: 'Security Analysis'
  dependsOn: Build
  condition: succeeded()
  jobs:
  - job: SonarQube
    steps:
    - task: SonarQubePrepare@5
      inputs:
        SonarQube: 'SonarQube-Service'
        scannerMode: 'CLI'
        configMode: 'manual'
        cliProjectKey: 'myapp'
        cliProjectName: 'My Application'
    
    - script: npm run sonar
      displayName: 'Run SonarQube analysis'
    
    - task: SonarQubePublish@5
      inputs:
        pollingTimeoutSec: '300'

- stage: DeployToStaging
  displayName: 'Deploy to Staging'
  dependsOn: SecurityScan
  condition: and(succeeded(), eq(variables['Build.SourceBranch'], 'refs/heads/main'))
  jobs:
  - deployment: Deploy
    displayName: 'Deploy to AKS'
    pool:
      vmImage: 'ubuntu-latest'
    environment: 'staging'
    strategy:
      runOnce:
        deploy:
          steps:
          - task: DownloadBuildArtifacts@1
            inputs:
              buildType: 'current'
              downloadType: 'single'
              artifactName: 'drop'
              downloadPath: '$(System.ArtifactsDirectory)'

          - task: AzureKeyVault@2
            inputs:
              azureSubscription: 'Azure-Service-Connection'
              KeyVaultName: 'myapp-keyvault'
              SecretsFilter: '*'
              RunAsPreJob: false

          - task: HelmDeploy@0
            inputs:
              connectionType: 'Azure Resource Manager'
              azureSubscription: 'Azure-Service-Connection'
              azureResourceGroup: 'myapp-rg'
              kubernetesCluster: 'myapp-aks'
              namespace: 'staging'
              command: 'upgrade'
              chartType: 'FilePath'
              chartPath: '$(Build.SourcesDirectory)/helm/myapp'
              releaseName: 'myapp'
              overrideValues: 'image.tag=$(tag)'
              waitForExecution: true

- stage: DeployToProduction
  displayName: 'Deploy to Production'
  dependsOn: DeployToStaging
  condition: succeeded()
  jobs:
  - deployment: Deploy
    displayName: 'Deploy to Production AKS'
    pool:
      vmImage: 'ubuntu-latest'
    environment: 'production'
    strategy:
      canary:
        increments: [25, 50, 100]
        preDeploy:
          steps:
          - script: echo "Pre-deployment checks..."
        deploy:
          steps:
          - task: HelmDeploy@0
            inputs:
              namespace: 'production'
              command: 'upgrade'
              releaseName: 'myapp'
              overrideValues: 'image.tag=$(tag)'
        on:
          failure:
            steps:
            - script: echo "Rollback initiated..."
          success:
            steps:
            - script: echo "Deployment successful..."
```

### Key Capabilities

**Azure Resource Manager Integration:**
Native deployment of ARM templates and Bicep files:

```yaml
- task: AzureResourceManagerTemplateDeployment@3
  inputs:
    deploymentScope: 'Resource Group'
    azureResourceManagerConnection: 'Azure-Service-Connection'
    subscriptionId: '00000000-0000-0000-0000-000000000000'
    action: 'Create Or Update Resource Group'
    resourceGroupName: 'myapp-rg'
    location: 'East US'
    templateLocation: 'Linked artifact'
    csmFile: '$(Build.SourcesDirectory)/infrastructure/main.bicep'
    csmParametersFile: '$(Build.SourcesDirectory)/infrastructure/parameters.json'
    deploymentMode: 'Incremental'
```

**Multi-Stage YAML with Approvals:**
Built-in environment approvals and checks:

```yaml
jobs:
- deployment: Production
  environment: 
    name: Production
    resourceName: myapp-api
  strategy:
    runOnce:
      deploy:
        steps:
        - script: echo "Deploying to production..."
```

**Variable Groups and Key Vault Integration:**
Secure secret management with Azure Key Vault:

```yaml
variables:
- group: my-variable-group
- name: mySecret
  value: $(mySecretFromKeyVault)  # Fetched from linked Key Vault
```

## 28.3 AWS CodePipeline

AWS CodePipeline provides native orchestration within the AWS ecosystem, integrating tightly with CodeCommit, CodeBuild, CodeDeploy, and third-party tools through Lambda integrations.

### Pipeline Structure

CodePipeline defines workflows through JSON or CloudFormation/Terraform, emphasizing stage-based progression with action types:

```json
{
  "pipeline": {
    "name": "MyAppPipeline",
    "roleArn": "arn:aws:iam::123456789012:role/CodePipelineRole",
    "artifactStore": {
      "type": "S3",
      "location": "codepipeline-artifacts-bucket"
    },
    "stages": [
      {
        "name": "Source",
        "actions": [
          {
            "name": "Source",
            "actionTypeId": {
              "category": "Source",
              "owner": "AWS",
              "provider": "CodeCommit",
              "version": "1"
            },
            "configuration": {
              "RepositoryName": "myapp",
              "BranchName": "main",
              "PollForSourceChanges": "false"
            },
            "outputArtifacts": [
              {
                "name": "SourceCode"
              }
            ]
          }
        ]
      },
      {
        "name": "Build",
        "actions": [
          {
            "name": "BuildAndTest",
            "actionTypeId": {
              "category": "Build",
              "owner": "AWS",
              "provider": "CodeBuild",
              "version": "1"
            },
            "configuration": {
              "ProjectName": "myapp-build"
            },
            "inputArtifacts": [
              {
                "name": "SourceCode"
              }
            ],
            "outputArtifacts": [
              {
                "name": "BuildArtifact"
              }
            ]
          }
        ]
      },
      {
        "name": "Deploy",
        "actions": [
          {
            "name": "DeployToECS",
            "actionTypeId": {
              "category": "Deploy",
              "owner": "AWS",
              "provider": "ECS",
              "version": "1"
            },
            "configuration": {
              "ClusterName": "production-cluster",
              "ServiceName": "myapp-service",
              "FileName": "imagedefinitions.json"
            },
            "inputArtifacts": [
              {
                "name": "BuildArtifact"
              }
            ]
          }
        ]
      }
    ]
  }
}
```

### CodeBuild Buildspec

AWS CodeBuild executes builds defined in `buildspec.yml`:

```yaml
version: 0.2

env:
  variables:
    NODE_VERSION: "20"
  parameter-store:
    DOCKER_REGISTRY_PASSWORD: /myapp/docker/password
  secrets-manager:
    DATABASE_URL: myapp-secret:database_url

phases:
  install:
    runtime-versions:
      nodejs: 20
    commands:
      - npm ci
      
  pre_build:
    commands:
      - echo Logging in to Amazon ECR...
      - aws ecr get-login-password --region $AWS_DEFAULT_REGION | docker login --username AWS --password-stdin $AWS_ACCOUNT_ID.dkr.ecr.$AWS_DEFAULT_REGION.amazonaws.com
      
  build:
    commands:
      - echo Build started on `date`
      - npm run build
      - npm test
      - docker build -t $IMAGE_REPO_NAME:$IMAGE_TAG .
      - docker tag $IMAGE_REPO_NAME:$IMAGE_TAG $AWS_ACCOUNT_ID.dkr.ecr.$AWS_DEFAULT_REGION.amazonaws.com/$IMAGE_REPO_NAME:$IMAGE_TAG
      
  post_build:
    commands:
      - echo Build completed on `date`
      - docker push $AWS_ACCOUNT_ID.dkr.ecr.$AWS_DEFAULT_REGION.amazonaws.com/$IMAGE_REPO_NAME:$IMAGE_TAG
      - printf '[{"name":"myapp","imageUri":"%s"}]' $AWS_ACCOUNT_ID.dkr.ecr.$AWS_DEFAULT_REGION.amazonaws.com/$IMAGE_REPO_NAME:$IMAGE_TAG > imagedefinitions.json

artifacts:
  files:
    - imagedefinitions.json
    - dist/**/*
  discard-paths: no
  
cache:
  paths:
    - node_modules/**/*
    - /root/.npm/**/*
```

### Key Characteristics

**CloudFormation/Terraform Integration:**
Native infrastructure-as-code deployment stages:

```yaml
- Name: DeployInfrastructure
  Actions:
  - Name: CreateChangeSet
    ActionTypeId:
      Category: Deploy
      Owner: AWS
      Provider: CloudFormation
      Version: 1
    Configuration:
      ActionMode: CHANGE_SET_REPLACE
      StackName: myapp-stack
      ChangeSetName: myapp-changeset
      TemplatePath: BuildArtifact::template.yaml
      Capabilities: CAPABILITY_IAM
```

**Lambda-Based Custom Actions:**
Extend pipelines with serverless functions for custom logic:

```python
# Lambda function for approval notification
import boto3

def lambda_handler(event, context):
    codepipeline = boto3.client('codepipeline')
    
    # Send SNS notification
    sns = boto3.client('sns')
    sns.publish(
        TopicArn='arn:aws:sns:us-east-1:123456789012:approvals',
        Message=f"Pipeline {event['pipeline']} requires approval"
    )
    
    return codepipeline.put_job_success_result(jobId=event['CodePipeline.job']['id'])
```

## 28.4 Google Cloud Build

Google Cloud Build executes builds in a serverless environment with native integration to Google Cloud Platform services, emphasizing container-centric workflows and build provenance.

### Build Configuration

Cloud Build uses `cloudbuild.yaml` to define build steps using container images:

```yaml
# cloudbuild.yaml
steps:
  # Build step using Docker
  - name: 'gcr.io/cloud-builders/docker'
    args: ['build', '-t', 'gcr.io/$PROJECT_ID/myapp:$SHORT_SHA', '.']
  
  # Push to Google Container Registry
  - name: 'gcr.io/cloud-builders/docker'
    args: ['push', 'gcr.io/$PROJECT_ID/myapp:$SHORT_SHA']
  
  # Run tests in the built image
  - name: 'gcr.io/$PROJECT_ID/myapp:$SHORT_SHA'
    args: ['npm', 'test']
    env:
      - 'NODE_ENV=test'
  
  # Security scanning with Container Analysis
  - name: 'gcr.io/cloud-builders/gcloud'
    args: ['artifacts', 'docker', 'images', 'scan', 'gcr.io/$PROJECT_ID/myapp:$SHORT_SHA']
  
  # Deploy to Cloud Run
  - name: 'gcr.io/google.com/cloudsdktool/cloud-sdk'
    entrypoint: gcloud
    args:
      - 'run'
      - 'deploy'
      - 'myapp'
      - '--image=gcr.io/$PROJECT_ID/myapp:$SHORT_SHA'
      - '--region=us-central1'
      - '--platform=managed'
      - '--allow-unauthenticated'

# Substitutions (variables)
substitutions:
  _REGION: us-central1
  _SERVICE_NAME: myapp

# Artifacts to store
artifacts:
  objects:
    location: 'gs://myapp-build-artifacts/'
    paths: ['coverage/**', 'test-results/**']

# Options
options:
  machineType: 'N1_HIGHCPU_8'  # 8 vCPUs
  diskSizeGb: '50'
  logging: CLOUD_LOGGING_ONLY
  dynamic_substitutions: true

# Timeout
timeout: '1200s'  # 20 minutes

# Secrets
availableSecrets:
  secretManager:
    - versionName: projects/$PROJECT_ID/secrets/api-key/versions/latest
      env: 'API_KEY'
```

### Build Triggers

Cloud Build supports triggers based on Git repository events:

```bash
# Create build trigger via gcloud
gcloud builds triggers create github \
  --repo-name=myapp \
  --repo-owner=myorg \
  --branch-pattern='^main$' \
  --build-config=cloudbuild.yaml \
  --name=main-trigger

# Create trigger for pull requests
gcloud builds triggers create github \
  --repo-name=myapp \
  --repo-owner=myorg \
  --pull-request-pattern='^main$' \
  --build-config=cloudbuild.yaml \
  --name=pr-trigger \
  --comment-control=COMMENTS_ENABLED  # Require /gcbrun comment
```

### Unique Features

**Source Provenance and Binary Authorization:**
Automatic generation of attestations for supply chain security:

```yaml
steps:
  # Build
  - name: 'gcr.io/cloud-builders/docker'
    args: ['build', '-t', 'gcr.io/$PROJECT_ID/myapp:$SHORT_SHA', '.']
  
  # Sign the image
  - name: 'gcr.io/cloud-builders/gcloud'
    args:
      - 'beta'
      - 'container'
      - 'binauthz'
      - 'attestations'
      - 'sign-and-create'
      - '--artifact-url=gcr.io/$PROJECT_ID/myapp:$SHORT_SHA'
      - '--attestor=projects/$PROJECT_ID/attestors/build-attestor'
      - '--keyversion=projects/$PROJECT_ID/locations/global/keyRings/signing/cryptoKeys/key1/cryptoKeyVersions/1'

images:
  - 'gcr.io/$PROJECT_ID/myapp:$SHORT_SHA'
```

**Buildpacks Integration:**
Automatic containerization without Dockerfile:

```yaml
steps:
  - name: 'gcr.io/buildpacks/builder:v1'
    args:
      - build
      - 'gcr.io/$PROJECT_ID/myapp:$SHORT_SHA'
      - '--builder=gcr.io/buildpacks/builder:v1'
images:
  - 'gcr.io/$PROJECT_ID/myapp:$SHORT_SHA'
```

## 28.5 Tekton

Tekton is a Kubernetes-native CI/CD framework that defines pipelines as Custom Resource Definitions (CRDs), enabling GitOps workflows and cloud-native scalability.

### Core Concepts

**Tasks:** Reusable, loosely coupled units of work:
```yaml
# task.yaml
apiVersion: tekton.dev/v1beta1
kind: Task
metadata:
  name: build-push-image
spec:
  params:
    - name: imageUrl
      type: string
      description: Image URL to push
    - name: imageTag
      type: string
      description: Image tag
  workspaces:
    - name: source
  steps:
    - name: build-and-push
      image: gcr.io/kaniko-project/executor:latest
      command:
        - /kaniko/executor
      args:
        - --dockerfile=$(workspaces.source.path)/Dockerfile
        - --destination=$(params.imageUrl):$(params.imageTag)
        - --context=$(workspaces.source.path)
      env:
        - name: DOCKER_CONFIG
          value: /tekton/home/.docker
```

**Pipelines:** Ordered collection of tasks:
```yaml
# pipeline.yaml
apiVersion: tekton.dev/v1beta1
kind: Pipeline
metadata:
  name: build-and-deploy
spec:
  workspaces:
    - name: shared-workspace
    - name: docker-config
  params:
    - name: git-url
      type: string
    - name: image-reference
      type: string
  tasks:
    - name: fetch-source
      taskRef:
        name: git-clone
      params:
        - name: url
          value: $(params.git-url)
      workspaces:
        - name: output
          workspace: shared-workspace
    
    - name: run-tests
      taskRef:
        name: npm-test
      runAfter:
        - fetch-source
      workspaces:
        - name: source
          workspace: shared-workspace
    
    - name: build-image
      taskRef:
        name: build-push-image
      runAfter:
        - run-tests
      params:
        - name: imageUrl
          value: $(params.image-reference)
        - name: imageTag
          value: $(tasks.fetch-source.results.commit)
      workspaces:
        - name: source
          workspace: shared-workspace
```

**PipelineRuns:** Instantiation of pipelines:
```yaml
# pipelinerun.yaml
apiVersion: tekton.dev/v1beta1
kind: PipelineRun
metadata:
  name: build-and-deploy-run
spec:
  pipelineRef:
    name: build-and-deploy
  params:
    - name: git-url
      value: https://github.com/org/myapp.git
    - name: image-reference
      value: gcr.io/project/myapp
  workspaces:
    - name: shared-workspace
      volumeClaimTemplate:
        spec:
          accessModes:
            - ReadWriteOnce
          resources:
            requests:
              storage: 1Gi
    - name: docker-config
      secret:
        secretName: regcred
```

### Triggers and Event Listeners

Tekton Triggers enable event-driven pipeline execution:

```yaml
# trigger-binding.yaml
apiVersion: triggers.tekton.dev/v1beta1
kind: TriggerBinding
metadata:
  name: github-push-binding
spec:
  params:
    - name: gitrevision
      value: $(body.head_commit.id)
    - name: gitrepositoryurl
      value: $(body.repository.url)
```

```yaml
# event-listener.yaml
apiVersion: triggers.tekton.dev/v1beta1
kind: EventListener
metadata:
  name: github-listener
spec:
  serviceAccountName: tekton-triggers
  triggers:
    - name: github-push
      interceptors:
        - ref:
            name: github
          params:
            - name: secretRef
              value:
                secretName: github-secret
                secretKey: secretToken
      bindings:
        - ref: github-push-binding
      template:
        ref: build-pipeline-template
```

### Tekton Dashboard and CLI

```bash
# Install Tekton CLI
brew install tektoncd-cli

# View pipelines
tkn pipeline list

# View pipeline runs
tkn pipelinerun logs -f build-and-deploy-run

# Start pipeline
tkn pipeline start build-and-deploy \
  --showlog \
  -p git-url=https://github.com/org/myapp.git \
  -p image-reference=gcr.io/project/myapp \
  -w name=shared-workspace,volumeClaimTemplateFile=workspace-template.yaml
```

## 28.6 Concourse CI

Concourse CI implements a resource-oriented philosophy where pipelines are configured as code (YAML) and resources represent external dependencies (git repos, Docker images, S3 buckets) that trigger jobs when they change.

### Pipeline Structure

Concourse pipelines consist of resources, jobs, and tasks:

```yaml
# pipeline.yml
resource_types:
  - name: pull-request
    type: docker-image
    source:
      repository: teliaoss/github-pr-resource

resources:
  - name: myapp-repo
    type: git
    source:
      uri: https://github.com/org/myapp.git
      branch: main
      username: ((git-username))
      password: ((git-password))

  - name: myapp-pull-requests
    type: pull-request
    source:
      repository: org/myapp
      access_token: ((github-access-token))

  - name: myapp-image
    type: docker-image
    source:
      repository: myregistry/myapp
      username: ((registry-username))
      password: ((registry-password))

  - name: version
    type: semver
    source:
      driver: git
      uri: https://github.com/org/myapp.git
      branch: version
      file: version

jobs:
  - name: run-tests
    plan:
      - get: myapp-repo
        trigger: true
      - task: run-tests
        config:
          platform: linux
          image_resource:
            type: docker-image
            source:
              repository: node
              tag: 20
          inputs:
            - name: myapp-repo
          run:
            dir: myapp-repo
            path: sh
            args:
              - -exc
              - |
                npm ci
                npm test

  - name: build-image
    plan:
      - get: myapp-repo
        passed: [run-tests]
        trigger: true
      - get: version
        params:
          bump: patch
      - put: myapp-image
        params:
          build: myapp-repo
          tag_file: version/version
          tag_as_latest: true
      - put: version
        params:
          file: version/version

  - name: deploy-to-staging
    plan:
      - get: myapp-image
        passed: [build-image]
        trigger: true
      - task: deploy
        config:
          platform: linux
          image_resource:
            type: docker-image
            source:
              repository: bitnami/kubectl
          params:
            KUBECONFIG: ((kubeconfig))
          run:
            path: sh
            args:
              - -c
              - |
                echo "$KUBECONFIG" > ~/.kube/config
                kubectl set image deployment/myapp app=myregistry/myapp:$(cat myapp-image/tag) -n staging
```

### Concourse Characteristics

**Resource Versioning:**
Every resource version is tracked; pipelines pin to specific versions and can be retriggered with old versions for reproducibility.

**Fly CLI:**
```bash
# Login
fly -t ci login -c http://concourse.company.com

# Set pipeline
fly -t ci set-pipeline -p myapp -c pipeline.yml -l credentials.yml

# Unpause pipeline
fly -t ci unpause-pipeline -p myapp

# Watch build
fly -t ci watch -j myapp/run-tests

# Hijack (debug) running container
fly -t ci hijack -j myapp/run-tests
```

## 28.7 Tool Comparison Matrix

| Feature | Jenkins | GitHub Actions | GitLab CI | CircleCI | Azure DevOps | AWS CodePipeline | Google Cloud Build | Tekton | Concourse |
|---------|---------|---------------|-----------|----------|--------------|------------------|-------------------|--------|-----------|
| **Hosting** | Self-hosted | SaaS/Enterprise | SaaS/Self-hosted | SaaS/Self-hosted | SaaS/Azure | AWS Native | GCP Native | Kubernetes | Self-hosted |
| **Config Format** | Groovy/YAML | YAML | YAML | YAML | YAML | JSON/YAML | YAML | YAML (CRDs) | YAML |
| **Kubernetes Native** | Plugin-based | No | Yes (Agent) | No | No | No | No | Yes | Yes |
| **Serverless** | No | Yes | Partial | Yes | Partial | Yes | Yes | No | No |
| **Secrets Mgmt** | Credentials Plugin | Secrets/Environments | Vault/Variables | Contexts/Projects | Key Vault/Groups | Secrets Manager | Secret Manager | Kubernetes Secrets | Vault/Cred Mgr |
| **Parallelization** | Agent-based | Matrix/Parallel | Parallel/Matrix | Resource Classes | Jobs/Stages | Stage-based | Step-based | Task parallelism | Resource checking |
| **Caching** | Plugin-based | Cache Action | Cache/Artifacts | Docker Layer Caching | Pipeline Caching | S3 Artifacts | GCS Cache | Workspace Volumes | Resource caching |
| **Pricing Model** | Infrastructure cost | Minutes-based | Per-user/Resource | Credits-based | Per-user | Per-action | Per-minute | Infrastructure | Infrastructure |
| **Best For** | Enterprise legacy | Open source projects | Integrated DevOps | Speed/Parallel | Microsoft shops | AWS workloads | GCP workloads | Cloud-native K8s | Resource-centric |

## 28.8 Migration Strategies

Migrating between CI/CD platforms requires careful planning to maintain pipeline continuity and historical context.

### Migration Approach

**1. Parallel Running (Phase 1):**
Run old and new pipelines simultaneously without blocking:

```yaml
# GitLab CI example during migration from Jenkins
stages:
  - test
  - migrate-verify

test:new:
  stage: test
  script:
    - npm test
  allow_failure: true  # Don't block while validating

test:legacy:
  stage: test
  script:
    - curl -X POST $JENKINS_URL/job/test/build
  rules:
    - if: $CI_PIPELINE_SOURCE == "merge_request_event"
```

**2. Feature Flag Migration:**
Use conditional logic to route builds:

```bash
#!/bin/bash
# migration-router.sh
if [ "$USE_NEW_CI" == "true" ]; then
  ./run-github-actions-local.sh
else
  ./trigger-jenkins.sh
fi
```

**3. Configuration Translation:**
Automated conversion tools:
- **Jenkins to GitHub Actions:** `jenkins-to-actions` converters
- **CircleCI to GitLab:** YAML restructuring with regex replacements
- **Generic:** Custom scripts using `yq` for YAML transformation

### Data Migration Considerations

**Artifacts and History:**
- Export build history from old system (if required for compliance)
- Migrate critical artifacts to new storage (S3, GCS, etc.)
- Document pipeline run IDs for audit trails

**Secrets Rotation:**
Mandatory when migrating platforms:
1. Audit all credentials in old system
2. Generate new credentials (don't copy values)
3. Store in new system with appropriate access controls
4. Revoke old credentials after verification

**Training and Documentation:**
- Create side-by-side comparison guides
- Document new failure modes and debugging procedures
- Establish "CI champions" for team support

---

## Chapter Summary and Preview

In this chapter, we explored alternative CI/CD platforms beyond the "big three," each optimized for specific operational contexts. CircleCI excels in developer experience and intelligent test parallelization, offering granular resource classes and sophisticated Docker layer caching for speed-optimized workflows. Azure DevOps Pipelines provides enterprise-grade integration with Microsoft ecosystems, featuring ARM template deployment, Azure Key Vault integration, and environment-based deployment strategies with built-in approval gates. AWS CodePipeline delivers native cloud orchestration within AWS, leveraging CodeBuild for serverless execution and integrating seamlessly with ECS, CodeDeploy, and CloudFormation for infrastructure-as-code deployment. Google Cloud Build emphasizes container-centric serverless execution with automatic provenance attestation and Binary Authorization integration for supply chain security. Tekton represents the Kubernetes-native approach, treating pipelines as CRDs enabling GitOps workflows and cloud-native scalability through Custom Resource Definitions. Concourse CI implements a resource-oriented philosophy emphasizing versioning and reproducibility through resource checking and explicit dependency graphs. The comparison matrix highlighted trade-offs between hosting models, configuration complexity, and cloud integration depth, while migration strategies emphasized parallel execution, secrets rotation, and phased transition approaches to minimize disruption.

**Key Takeaways:**
- Select CircleCI when pipeline speed and test parallelization are paramount, leveraging resource classes and timing-based test splitting for optimal performance
- Choose Azure DevOps for Microsoft-centric enterprises requiring deep Azure service integration, ARM template deployment, and enterprise identity management through Entra ID
- Implement AWS CodePipeline for AWS-native architectures to minimize cross-cloud data transfer costs and leverage native ECS/EKS deployment actions
- Adopt Tekton for Kubernetes-native GitOps workflows where pipelines must be treated as versioned infrastructure alongside application deployments
- Execute platform migrations using parallel running strategies with feature flags, never performing "big bang" transitions, and always rotating secrets rather than transferring them between platforms
- Consider Concourse CI for teams prioritizing resource versioning and reproducibility over convenience, particularly in regulated industries requiring exact build traceability

**Next Chapter Preview:**
Chapter 29: CD Fundamentals transitions from continuous integration to continuous deployment, examining the philosophical and practical distinctions between CI, CD, and Continuous Delivery. We will explore deployment pipeline architecture, environment promotion strategies, and the cultural requirements for implementing true continuous deployment where every commit automatically flows to production. This chapter establishes the foundation for deployment strategies including blue-green, canary, and rolling updates detailed in subsequent chapters, bridging the gap between building artifacts and safely delivering them to users.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='27. gitlab_cicd.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../6. continuous_deployment/29. cd_fundamentals.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
